In [1]:
stock = 'AMD'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-23    16.660000
2018-07-24    16.190001
2018-07-25    16.049999
2018-07-26    18.350000
2018-07-27    18.940001
2018-07-30    19.420000
2018-07-31    18.330000
2018-08-01    18.480000
2018-08-02    18.790001
2018-08-03    18.490000
2018-08-06    19.430000
2018-08-07    19.559999
2018-08-08    19.580000
2018-08-09    19.100000
2018-08-10    19.059999
2018-08-13    19.730000
2018-08-14    20.020000
2018-08-15    19.700001
2018-08-16    19.330000
2018-08-17    19.770000
2018-08-20    19.980000
2018-08-21    20.400000
2018-08-22    20.900000
2018-08-23    22.290001
2018-08-24    23.980000
2018-08-27    25.260000
2018-08-28    25.049999
2018-08-29    25.200001
2018-08-30    24.889999
2018-08-31    25.170000
                ...    
2018-09-18    31.930000
2018-09-19    31.209999
2018-09-20    31.180000
2018-09-21    31.020000
2018-09-24    32.610001
2018-09-25    32.570000
2018-09-26    32.189999
2018-09-27    32.590000
2018-09-28    30.889999
2018-10-01    31.420000
2018-10-02 

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


23.506735757824238
[24.4078026  25.3958036  21.87920804 24.09160308 21.75926148]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")